# 🎓 Notebook 09: Semantic Chunking (High Precision)

Traditional chunking (like fixed-size windows) often breaks context in the middle of a sentence or a logical thought. **Semantic Chunking** uses AI to find the "natural breaks" in a document.

### Learning Objectives:
1. Understand the limitations of fixed-size chunking.
2. Implement a **Similarity-Based Splitter** from scratch.
3. Compare Semantic vs. Fixed chunking results.

In [ ]:
from typing import List
import numpy as np
from src.core.bootstrap import get_container
from src.application.services.chunking import count_tokens

container = get_container()
embedder = container["embeddings"]

## 1. The Strategy: Sentence Similarity

We split the text into sentences, embed them, and then calculate the "distance" between adjacent sentences. When the distance crosses a threshold, we know the topic has changed.

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def semantic_split(text: str, threshold: float = 0.8):
    # 1. Split into sentences (simple split for demo)
    sentences = [s.strip() for s in text.split('.') if len(s.strip()) > 10]
    
    # 2. Embed sentences
    embeddings = [embedder.embed_one(s) for s in sentences]
    
    chunks = []
    current_chunk = [sentences[0]]
    
    for i in range(len(sentences) - 1):
        sim = cosine_similarity(embeddings[i], embeddings[i+1])
        
        if sim < threshold:
            # Topic shift detected!
            chunks.append(". ".join(current_chunk) + ".")
            current_chunk = [sentences[i+1]]
        else:
            current_chunk.append(sentences[i+1])
            
    chunks.append(". ".join(current_chunk) + ".")
    return chunks

## 2. Live Test

Notice how the semantic chunker groups the sentences by theme rather than character count.

In [ ]:
sample_text = """
The sun is a star at the center of the solar system. It is a nearly perfect sphere of hot plasma.
Quantum mechanics is a fundamental theory in physics. It describes the physical properties of nature at the scale of atoms.
Baking a cake requires flour, eggs, and sugar. You must preheat the oven to 350 degrees.
"""

semantic_chunks = semantic_split(sample_text, threshold=0.6)

for i, chunk in enumerate(semantic_chunks):
    print(f"--- Chunk {i+1} ---")
    print(chunk)
    print("\n")

## 3. Why this matters for RAG

By keeping semantically related sentences together:
1. **Better Retrieval**: The vector for the chunk is more "focused."
2. **Lower Costs**: You don't retrieve irrelevant half-sentences from adjacent topics.
3. **Higher Accuracy**: The LLM has perfect context for the specific topic.